### 1.  OBJETIVO DO CÓDIGO

Calcular automaticamente quanto você precisa repor de cada produto em cada loja, semana a semana, para nunca faltar estoque.

In [ ]:
#ativar ambiente -> .venv\Scripts\activate.bat
#desativar ambiente -> deactivate
from pathlib import Path
import pandas as pd
import numpy as np


DATA_DIR = Path("data")
SAIDA_DIR = Path("outputs")
SAIDA_DIR.mkdir(exist_ok=True)

ARQUIVO_VENDAS = "forecast_ano_v03.xlsx"
ABA_VENDAS = "BD"
ARQUIVO_ESTOQUE = DATA_DIR / "estoque-semanal-total.xlsx"

SEMANA_INICIO = "W34_25"
SEMANAS_ALVO = 10
         # Quantas semanas olhar para frente

In [ ]:
# ==================================================
# 1. FUNÇÃO: CARREGAR VENDAS (AGORA POR ARTIGO-COR)
# ==================================================
def carregar_vendas(caminho_arquivo, nome_aba):
    """Carrega e processa arquivo de vendas por Artigo-Cor"""
    print("📊 Carregando vendas por Artigo-Cor...")
    
    # Carrega dados
    df = pd.read_excel(caminho_arquivo, sheet_name=nome_aba, engine='openpyxl') #Tradução: "Abra o arquivo Excel na aba especificada"
    df.columns = df.columns.str.strip() #Tradução: "Remove espaços extras dos nomes das colunas"
    
    # Extrai código da filial (primeiro número antes do espaço)
    df["FILIAL"] = df["FILIAL"].astype(str).str.split().str[0] #Tradução: "Pega a parte antes do primeiro espaço"
    df["FILIAL"] = pd.to_numeric(df["FILIAL"], errors="coerce").fillna(0).astype(int) #Tradução: "Converte para número, se não der certo, vira 0"
    
    # Processa produtos e cores
    df["PRODUTO"] = df["PRODUTO"].astype(str).str.upper().str.strip() # Tradução: "Deixe PRODUTO e COR em maiúsculas e sem espaços extras"
    df["COR"] = df["COR"].astype(str).str.upper().str.strip()
    
    # Cria chave Artigo-Cor (ignorando tamanho)
    df["ArtigoCor"] = df["PRODUTO"]  + df["COR"]
    
    # Identifica colunas de semanas
    semanas_colunas = [col for col in df.columns if col.startswith('202') and '_W' in col] #"Encontre colunas que começam com '202' e têm '_W'", Exemplo: Encontra "2025_W34", "2025_W35", etc.
    
    # Renomeia semanas para formato padrão (WXX_YY)
    rename_map = {} #Como um "tradutor": {"nome_antigo": "nome_novo"}, Exemplo: {"2025_W34": "W34_25"}
    for col in semanas_colunas: #Exemplo do que está em semanas_colunas: # ["2025_W34", "2025_W35", ...]
        partes = col.split('_W') #Tradução: "Divida o nome usando '_W' como separador", "2025_W34""2025_W34".split('_W')["2025", "34"]
        if len(partes) == 2: # "Se conseguiu dividir em exatamente 2 partes..."
            ano = partes[0][-2:]  # Últimos 2 dígitos do ano, partes[0]:  "2025", Últimas 2 posições: 2 5, partes[0][-2:],  "25"
            semana = partes[1] #"Pegue a segunda parte (número da semana)"
            rename_map[col] = f"W{semana}_{ano}" # "Crie o novo nome no formato WXX_YY", Exemplo: "W34_25"
    
    df = df.rename(columns=rename_map)
    semanas = sorted(rename_map.values())
    
    # Agrupa por Artigo-Cor e Filial (ignorando tamanho)
    colunas_agrupar = ["FILIAL", "ArtigoCor"]
    df_agrupado = df[colunas_agrupar + semanas].groupby(colunas_agrupar, as_index=False).sum()
    
    print(f"✅ Vendas: {len(df_agrupado)} Artigos-Cor, {len(semanas)} semanas")
    return df_agrupado, semanas

In [ ]:
# ==================================================
# 2. FUNÇÃO: CARREGAR ESTOQUE 
# ==================================================
def carregar_estoque(caminho_arquivo):
    """Carrega estoque apenas de filiais franquiadas por Artigo-Cor"""
    print("📦 Carregando estoque de franquias por Artigo-Cor...")
    
    df = pd.read_excel(caminho_arquivo, engine='openpyxl')
    df.columns = df.columns.str.strip()
    
    # VERIFICAÇÃO DAS COLUNAS EXISTENTES
    print("📋 Colunas no estoque:", df.columns.tolist())
    
    # Padroniza nome das colunas (se necessário)
    if "Ponto Venda Cód" in df.columns:
        df = df.rename(columns={"Ponto Venda Cód": "FILIAL"})
    if "Artigo" in df.columns:
        df = df.rename(columns={"Artigo": "ArtigoCor"})
    
    # Processa dados
    df["FILIAL"] = pd.to_numeric(df["FILIAL"], errors="coerce").fillna(0).astype(int) # "Converte FILIAL para numérico, substituindo erros por 0 e convertendo para inteiro"
    df["ArtigoCor"] = df["ArtigoCor"].astype(str).str.upper().str.strip() # "Deixa ArtigoCor em maiúsculas e sem espaços extras"
    
    # Agrupa estoque por Artigo-Cor e Filial
    colunas_agrupar = ["FILIAL", "ArtigoCor"]
    df_agrupado = df.groupby(colunas_agrupar, as_index=False)["Estoque Semanal Total"].sum() #"Some todo estoque por ArtigoCor+Filial"
    df_agrupado = df_agrupado.rename(columns={"Estoque Semanal Total": "ESTOQUE_INICIAL"})  # "Renomeia coluna de estoque para ESTOQUE_INICIAL"
    
    print(f"✅ Estoque: {len(df_agrupado)} Artigos-Cor de franquias") # "Exibe quantos Artigos-Cor de franquias foram encontrados no estoque"
    print("📋 Colunas após processamento:", df_agrupado.columns.tolist()) # O .tolist() é um método do pandas que converte uma Series ou Index (coluna do DataFrame) em uma lista Python comum.
    return df_agrupado

In [ ]:
# ==================================================
# 3. FUNÇÃO: CALCULAR ALVOS DE ESTOQUE
#  ALVO DE ESTOQUE = Quantidade ideal que você quer ter para cobrir as próximas X semanas de vendas.
# "Se eu sei que vou vender 100 unidades nas próximas 4 semanas, quero ter pelo menos 100 unidades em estoque"
# ==================================================
def calcular_alvos(df, todas_semanas, semana_inicio, semanas_alvo): #todas_semanas: ['W34_25', 'W35_25']
    """Calcula alvos de estoque para cada Artigo-Cor""" #semana_inicio: 'W34_25', semanas_alvo: 10
    print("🎯 Calculando alvos de estoque...") #semanas_alvo: 10
    
    # Encontra índice da semana inicial
    inicio_idx = todas_semanas.index(semana_inicio) #Tradução: "Em que posição da lista está minha semana inicial?" no caso de exemplo, 0
    semanas_simulacao = todas_semanas[inicio_idx:] #"Pegue todas as semanas a partir da semana inicial"
    # Resultado: da semana inicial até o final dos dados
    # Calcula alvo (soma das vendas das próximas X semanas)
    for semana in semanas_simulacao: #Tradução: "Para cada semana na lista de semanas a serem simuladas"
        idx_semana = todas_semanas.index(semana) #"Me diga em que posição está esse elemento", exemplo: 'W34_25' está na posição 0, 'W35_25' na posição 1, etc.
        semanas_somar = todas_semanas[idx_semana:idx_semana + semanas_alvo] ## todas_semanas[0:0+10] = todas_semanas[0:10], Tradução: "Para W34_25, some as vendas das próximas 10 semanas (incluindo ela mesma)"
        df[f"ALVO_{semana}"] = df[semanas_somar].sum(axis=1)  #"Crie coluna ALVO_W34_25 = soma das vendas de W34_25 até W43_25" e Operação horizontal (somar colunas) →
    #semanas_somar = todas_semanas[0:10]  # W34_25 até W43_25, ALVO_W34_25 = 10 + 15 + 12 + 8 + 20 + 18 + 22 + 16 + 14 + 10 = 145
    return df, semanas_simulacao
#
#"Na semana 34, eu preciso ter 145 unidades em estoque para cobrir as vendas das próximas 10 semanas (até a semana 43)"
# "Cada semana precisa ter estoque suficiente para cobrir as próximas 10 semanas"

In [ ]:
# ==================================================
# 4. FUNÇÃO: SIMULAR REPOSIÇÃO SEMANAL
# ==================================================
def simular_reposicao(df, semanas_simulacao):
    """Simula reposição semana a semana"""
    print("🔄 Simulando reposição...")
    
    coluna_estoque_anterior = "ESTOQUE_INICIAL" #"Comece usando o estoque atual (inicial) como ponto de partida"
    #Tradução: "Para cada semana da simulação, vou calcular quanto repor"
    for semana in semanas_simulacao: #semanas_simulacao = ['W34_25', 'W35_25', 'W36_25', 'W37_25', 'W38_25', 
        venda_semana = semana  ## 'W34_25'
        alvo_semana = f"ALVO_{semana}" # 'ALVO_W34_25' 
        reposicao_semana = f"REPOSICAO_{semana}" # 'REPOSICAO_W34_25'
        estoque_semana = f"ESTOQUE_{semana}" # 'ESTOQUE_W34_25'
        #Tradução: "Defina os nomes das colunas que vou usar/criar nesta semana"

        # Cálculo da reposição: quanto preciso repor para atingir o alvo
        df[reposicao_semana] = np.maximum(0, df[alvo_semana] - df[coluna_estoque_anterior] + df[venda_semana])
        #ALVO - ESTOQUE_ANTERIOR = Quanto me falta para atingir o alvo
        #+ VENDA = Mais o que vou vender nesta semana
        #máximo(0, ...) = Nunca repor quantidade negativa
        # Atualiza estoque: estoque anterior - vendas + reposição
        #Ex:Estoque Ideal (Alvo): 100 unidades (preciso ter para as próximas semanas)
        #Estoque Atual: 30 unidades (o que tenho agora)
        #Vendas desta semana: 10 unidades (o que vou vender)
        #100 - 30 = 70 + 10 = 80 unidades a repor, "Além das 70, preciso repor o que vou vender (10 unidades)"

        df[estoque_semana] = df[coluna_estoque_anterior] - df[venda_semana] + df[reposicao_semana]
        #"Como meu estoque se transforma do início para o final da semana, considerando vendas e reposições"
        #Estoque inicial: 30 unidades, Vendas da semana: 10 unidades, Reposição: 80 unidades
        #df[coluna_estoque_anterior] - df[venda_semana] = 30 - 10 = 20 unidades
        #+ df[reposicao_semana] = 20 + 80 = 100 unidades
        #ESTOQUE_W34_25 = 80 - 25 + 65
        
        # Prepara para próxima semana
        coluna_estoque_anterior = estoque_semana # "O estoque desta semana vira o 'estoque anterior' da próxima semana"
    #Após cálculos: coluna_estoque_anterior = "ESTOQUE_W34_25", W35_25:coluna_estoque_anterior = "ESTOQUE_W34_25" (resultado da semana anterior)
    return df

In [ ]:
# ==================================================
# 5. FUNÇÃO: SALVAR RESULTADOS (CORRIGIDA)
# ==================================================
def salvar_resultados(df, semanas_simulacao, pasta_saida): #semanas simulacao = ['W34_25', 'W35_25', 'W36_25', ..., 'W43_25']
    """Salva resultados em Excel"""
    print("💾 Salvando resultados...")
    
    # VERIFICAÇÃO DAS COLUNAS ANTES DE SALVAR
    print("📋 Colunas disponíveis para salvar:", df.columns.tolist()) #"Me mostre todas as colunas que existem no momento"
    
    # Seleciona colunas para exportar (CORRIGIDO - vírgula faltando)
    colunas_base = ["FILIAL", "ArtigoCor", "ESTOQUE_INICIAL"] # "Informações básicas para identificar cada linha"
    colunas_vendas = semanas_simulacao #Onde: semanas_simulacao = ['W34_25', 'W35_25', 'W36_25', ..., 'W43_25']
    colunas_alvos = [f"ALVO_{s}" for s in semanas_simulacao] #Resultado: ['ALVO_W34_25', 'ALVO_W35_25', 'ALVO_W36_25', ..., 'ALVO_W43_25']
    colunas_reposicao = [f"REPOSICAO_{s}" for s in semanas_simulacao] #Resultado: ['REPOSICAO_W34_25', 'REPOSICAO_W35_25', ..., 'REPOSICAO_W43_25']
    colunas_estoque = [f"ESTOQUE_{s}" for s in semanas_simulacao] #Resultado: ['ESTOQUE_W34_25', 'ESTOQUE_W35_25', ..., 'ESTOQUE_W43_25']
    
    todas_colunas = colunas_base + colunas_vendas + colunas_alvos + colunas_reposicao + colunas_estoque # "Todas as colunas que quero salvar no Excel"
    
    # Filtra apenas colunas que existem no DataFrame
    colunas_existentes = [col for col in todas_colunas if col in df.columns]  # "Verifica quais colunas estão realmente no DataFrame"
    # Exemplo: Se df.columns = ['FILIAL', 'ArtigoCor', 'ESTOQUE_INICIAL', 'W34_25', 'ALVO_W34_25', 'REPOSICAO_W34_25', 'ESTOQUE_W34_25'], então colunas_existentes = ['FILIAL', 'ArtigoCor', 'ESTOQUE_INICIAL', 'W34_25', 'ALVO_W34_25', 'REPOSICAO_W34_25', 'ESTOQUE_W34_25']
    print("📋 Colunas que serão salvas:", colunas_existentes)
    
    df_final = df[colunas_existentes] #"Crie nova tabela com apenas as colunas que quero salvar"
    
    # Cria pasta se não existir
    pasta_saida.mkdir(exist_ok=True) #"Crie a pasta de saída se ela não existir ainda"
    caminho_saida = pasta_saida / "projecao_artigo_cor.xlsx"
    df_final.to_excel(caminho_saida, index=False)
    
    print(f"✅ Resultados salvos em: {caminho_saida}")
    return df_final

In [ ]:
# ==================================================
# 6. FUNÇÃO PRINCIPAL
# ==================================================
def main():
    """Executa a simulação completa por Artigo-Cor"""
    print("="*60)
    print("🚀 SIMULAÇÃO DE REPOSIÇÃO POR ARTIGO-COR")
    print("="*60)
    
    try: #Tenta executar o código principal
        # 1. Carregar dados
        vendas, todas_semanas = carregar_vendas(ARQUIVO_VENDAS, ABA_VENDAS) #Chama duas funções para carregar os dados-"Carrega os dados de vendas por Artigo-Cor", 
        estoque = carregar_estoque(ARQUIVO_ESTOQUE) #estoque: Retorna DataFrame de estoque
        
        # 2. Combinar vendas com estoque (apenas franquias)
        print("\n🔗 Combinando dados...")
        dados_completos = pd.merge(
            vendas,
            estoque,
            on=["FILIAL", "ArtigoCor",], # "Combine os dados de vendas e estoque usando FILIAL e ArtigoCor como chaves"
            how="inner"  # Mantém apenas franquias com estoque
        )
        
        print(f"✅ Merge realizado: {len(dados_completos)} Artigos-Cor de franquias")
        
        # 3. Calcular alvos de estoque
        dados_com_alvos, semanas_simulacao = calcular_alvos(
            dados_completos, todas_semanas, SEMANA_INICIO, SEMANAS_ALVO # "Calcula alvos de estoque a partir da semana inicial e número de semanas alvo"
        )#SEMANA_INICIO = "W34_25": Começa na semana 34 de 2025
        #SEMANAS_ALVO = 10: Quer ter estoque para 10 semanas à frente
        # 4. Simular reposição
        dados_finais = simular_reposicao(dados_com_alvos, semanas_simulacao)
        
        # 5. Salvar resultados
        resultado = salvar_resultados(dados_finais, semanas_simulacao, SAIDA_DIR) # salva os resultados em um arquivo Excel
        
        print("="*60)
        print("🎉 SIMULAÇÃO CONCLUÍDA COM SUCESSO!")
        print(f"📈 Período: {semanas_simulacao[0]} a {semanas_simulacao[-1]}")  #Período: W34_25 a W38_25
        print(f"📊 Artigos-Cor analisados: {len(resultado)}")
        print(f"🏪 Filiais franquiadas: {resultado['FILIAL'].nunique()}")
        print("="*60)
        
    except Exception as e: #Se algo der errado, captura o erro e mostra detalhes
        print(f"❌ ERRO: {e}")
        import traceback # "Importa a biblioteca traceback para mostrar detalhes do erro"
        traceback.print_exc()

# Executar
if __name__ == "__main__": # "Se este arquivo for executado diretamente, chama a função principal"
    main()

🚀 SIMULAÇÃO DE REPOSIÇÃO POR ARTIGO-COR
📊 Carregando vendas por Artigo-Cor...
✅ Vendas: 79084 Artigos-Cor, 33 semanas
📦 Carregando estoque de franquias por Artigo-Cor...
📋 Colunas no estoque: ['FILIAL', 'ArtigoCor', 'Semana_365', 'Estoque Semanal Total']
✅ Estoque: 109988 Artigos-Cor de franquias
📋 Colunas após processamento: ['FILIAL', 'ArtigoCor', 'ESTOQUE_INICIAL']

🔗 Combinando dados...
✅ Merge realizado: 68355 Artigos-Cor de franquias
🎯 Calculando alvos de estoque...
🔄 Simulando reposição...
💾 Salvando resultados...
📋 Colunas disponíveis para salvar: ['FILIAL', 'ArtigoCor', 'W01_26', 'W02_26', 'W03_26', 'W04_26', 'W05_26', 'W06_26', 'W07_26', 'W08_26', 'W09_26', 'W10_26', 'W11_26', 'W12_26', 'W13_26', 'W14_26', 'W34_25', 'W35_25', 'W36_25', 'W37_25', 'W38_25', 'W39_25', 'W40_25', 'W41_25', 'W42_25', 'W43_25', 'W44_25', 'W45_25', 'W46_25', 'W47_25', 'W48_25', 'W49_25', 'W50_25', 'W51_25', 'W52_25', 'ESTOQUE_INICIAL', 'ALVO_W34_25', 'ALVO_W35_25', 'ALVO_W36_25', 'ALVO_W37_25', 'ALVO